<a href="https://colab.research.google.com/github/kuihua/NLP-Week1-Text-Classification/blob/colab-version/movie_review_sentiment_classifier_with_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Introduction

SMS Spam Detection using Transfer learning.

# Setup Python Libraries (pip)

In [2]:
#install some Python packages with pip

%pip install numpy torch datasets transformers evaluate --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.5 MB/s eta 0:00:00


In [3]:
# let's check the version we are using

%pip freeze | grep -E '^numpy|^torch|^datasets|^transformers|^evaluate'

datasets==4.0.0
evaluate==0.4.6
numpy==2.0.2
torch==2.9.0+cu126
torchao==0.10.0
torchaudio==2.9.0+cu126
torchdata==0.11.0
torchsummary==1.5.1
torchtune==0.6.1
torchvision==0.24.0+cu126
transformers==4.57.1


In [14]:
pip install scikit-learn matplotlib

# Importing Libraries

In [4]:
# load ds
from datasets import load_dataset
# tokenizer
from transformers import AutoTokenizer

In [5]:
import numpy as np
# training metrics
import evaluate

In [6]:
# training configs
import os
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, IntervalStrategy
from transformers import DataCollatorWithPadding

In [7]:
# to test examples
from transformers import pipeline
import torch

# Create SMS Dataset for Fine-tuning BERT

## Let's load the SMS Dataset

In [8]:
# path to dataset
path =  "/content/drive/MyDrive/nlp_archive/dataset.csv"

In [9]:
raw_dataset = load_dataset("csv", data_files=path)

Generating train split: 0 examples [00:00, ? examples/s]

## Let's create the train, validation, test sets

In [19]:
# get train and validation set

dataset = raw_dataset['train'].train_test_split(test_size=0.2, seed=42, shuffle=True)
dataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'text'],
        num_rows: 8768
    })
    test: Dataset({
        features: ['labels', 'text'],
        num_rows: 2193
    })
})

In [20]:
dataset = raw_dataset['train'].train_test_split(test_size=0.2, seed=42, shuffle=True)

In [21]:
# rename validation key to 'val'

dataset['val'] = dataset['test']
dataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'text'],
        num_rows: 8768
    })
    test: Dataset({
        features: ['labels', 'text'],
        num_rows: 2193
    })
    val: Dataset({
        features: ['labels', 'text'],
        num_rows: 2193
    })
})

In [ ]:
# from datasets import DatasetDict

# # Split raw_dataset['train'] into a training set (80%) and a temporary set (20%)
# # The temporary set will then be further split into validation and test sets.
# train_temp_split = raw_dataset['train'].train_test_split(test_size=0.2, seed=42, shuffle=True)

# # Split the temporary set (20%) into validation (10%) and test (10%)
# # We take 50% of the temporary set for validation and 50% for test.
# val_test_split = train_temp_split['test'].train_test_split(test_size=0.5, seed=42, shuffle=True)

# # Assemble the final DatasetDict with distinct train, validation, and test sets
# dataset = DatasetDict({
#     'train': train_temp_split['train'],
#     'val': val_test_split['train'], # Renaming 'train' from val_test_split to 'val'
#     'test': val_test_split['test']
# })

# dataset

## We start by tokenizing our dataset with the BERT's Fast Tokenizer

In [ ]:
# let's import the pretrained faster tokenizer from huggingface
# source: (https://huggingface.co/distilbert-base-uncased)

In [12]:
checkpoint = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint, use_fast=True)
tokenizer

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

DistilBertTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [22]:
id2label = {0:'ham',1:'spam'}
label2id = {'ham':0,'spam':1}

# Convert string labels ('ham', 'spam') to integer labels (0, 1) using the label2id mapping.
def encode_labels(example):
    example['labels'] = label2id[example['labels']]
    return example

# process one label at a time
dataset = dataset.map(encode_labels, batched=False)

# tokenize the text in batches with truncation and padding based on BERT requirements
def tokenization(example):
    # Convert all elements in 'example["text"]' to strings, handling None or NaN values
    texts = [str(t) if t is not None and not (isinstance(t, float) and np.isnan(t)) else "" for t in example['text']]
    return tokenizer(texts, truncation=True, padding=True)

tokenized_dataset = dataset.map(tokenization, batched=True, remove_columns=['text'])
tokenized_dataset

Map:   0%|          | 0/8768 [00:00<?, ? examples/s]

Map:   0%|          | 0/2193 [00:00<?, ? examples/s]

Map:   0%|          | 0/8768 [00:00<?, ? examples/s]

Map:   0%|          | 0/2193 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 8768
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 2193
    })
    val: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 2193
    })
})

# Setup Training Metrics (Accuracy, F1)

In [16]:
# we setup the training to evaluate the accuracy and f1 scores
accuracy_metric = evaluate.load('accuracy')
f1_metric = evaluate.load('f1')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels)
    return {**accuracy, **f1}

# Setup Training Configurations

In [17]:
# get bert model with a sequence classification head for text classification
# source: (https://huggingface.co/distilbert-base-uncased)
checkpoint = 'distilbert-base-uncased'
num_labels = 2 # ham/spam

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=num_labels, id2label=id2label, label2id=label2id)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
# setup custom training arguments
# 1. store training checkpoints to 'results' output directory
# 2. fine-tune for just 1 epoch
# 3,4. use 16 as a batch size to speed things up
# 5. evaluate validation set every 500 steps (this is the default steps)
# 6. load the best model based on the lowest validation loss at the end of training

training_args = TrainingArguments(
    seed=42,
    output_dir = './results',
    num_train_epochs = 1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    load_best_model_at_end=True,
    eval_strategy = "epoch",
    save_strategy = 'epoch',
    # 0.00005 is default
    learning_rate = 0.00005,
    # turns off wandb
    report_to=None
)

# disable wandb logging (a v4 huggingface artifact)
# following line is deprecated
os.environ['WANDB_DISABLED']= "true"

# Ensure the dataset is in PyTorch format for the Trainer to correctly pass labels to the model for loss computation.
tokenized_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

# Instantiate DataCollatorWithPadding to ensure labels are correctly passed
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True, return_tensors="pt")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['val'],
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


# Evaluate UnFine-Tuned BERT on Test Set for a Baseline Metric


In [28]:
# # let's first evaluate unfine-tuned model with test set
trainer.evaluate(tokenized_dataset['test'])

{'eval_loss': 0.6357584595680237,
 'eval_model_preparation_time': 0.0015,
 'eval_accuracy': 0.7829457364341085,
 'eval_f1': 0.0,
 'eval_runtime': 35.6655,
 'eval_samples_per_second': 61.488,
 'eval_steps_per_second': 3.869}

Without fine-tuning BERT, our model currently has around **78.3% Accuracy (eval_accuracy)** and **0% F1 (eval_f1)**, which is pretty bad.


Let's make it better with transfer learning!

# Fine-Tune BERT with SMS Dataset

First run.

In [29]:
trainer.train()

Epoch,Training Loss,Validation Loss,Model Preparation Time,Accuracy,F1
1,0.121800,0.082801,0.001500,0.977656,0.948148


TrainOutput(global_step=548, training_loss=0.11842332965266096, metrics={'train_runtime': 449.7357, 'train_samples_per_second': 19.496, 'train_steps_per_second': 1.218, 'total_flos': 1161474151415808.0, 'train_loss': 0.11842332965266096, 'epoch': 1.0})

In [30]:
# let's see how well it did in the test set

trainer.evaluate(tokenized_dataset['test'])

{'eval_loss': 0.08280070871114731,
 'eval_model_preparation_time': 0.0015,
 'eval_accuracy': 0.97765617875057,
 'eval_f1': 0.9481481481481482,
 'eval_runtime': 36.2678,
 'eval_samples_per_second': 60.467,
 'eval_steps_per_second': 3.805,
 'epoch': 1.0}

**WOAH!** We got a **97.8% Accuracy (eval_accuracy)** and **94.8% F1 (eval_f1)** with just **1 epoch**! 🤯

# Fine Tuning Hyperparameters

Second run.

In [31]:
# Changing hyperparameter(s)

training_args = TrainingArguments(
    seed=42,
    output_dir = './results',
    num_train_epochs = 1,
    # batch size 16 -> 32
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    load_best_model_at_end=True,
    eval_strategy = "epoch",
    save_strategy = 'epoch',
    learning_rate = 0.00005,
    # turns off wandb
    report_to=None
)

# disable wandb logging (a v4 huggingface artifact)
# following line is deprecated
os.environ['WANDB_DISABLED']= "true"

# Ensure the dataset is in PyTorch format for the Trainer to correctly pass labels to the model for loss computation.
tokenized_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

# Instantiate DataCollatorWithPadding to ensure labels are correctly passed
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True, return_tensors="pt")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['val'],
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [32]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.068905,0.981760,0.957983


TrainOutput(global_step=274, training_loss=0.047809423321354995, metrics={'train_runtime': 441.2682, 'train_samples_per_second': 19.87, 'train_steps_per_second': 0.621, 'total_flos': 1161474151415808.0, 'train_loss': 0.047809423321354995, 'epoch': 1.0})

In [33]:
trainer.evaluate(tokenized_dataset['test'])

{'eval_loss': 0.06890489161014557,
 'eval_accuracy': 0.9817601459188326,
 'eval_f1': 0.957983193277311,
 'eval_runtime': 35.5123,
 'eval_samples_per_second': 61.753,
 'eval_steps_per_second': 1.943,
 'epoch': 1.0}

# Try out some examples!

In [ ]:
from transformers import pipeline
import torch

In [ ]:
# get current device with pytorch
device = torch.cuda.current_device()

# create pipeline for sentiment classifier with custom model and tokenizer
spam_classifier = pipeline(task='text-classification', model=model, tokenizer=tokenizer,device=device)

Device set to use cuda:0


In [ ]:
msg = """
hi friend long time no see lets go hang out
at like the same place as last time
"""
spam_classifier(msg)

[{'label': 'ham', 'score': 0.9990553259849548}]

# Resources

### If you would like to use this model without running the entire notebook, try the model at my [HuggingFace](https://huggingface.co/wesleyacheng/movie-review-sentiment-classifier-with-bert).

### If you woud like to get this in GitHub, here's my [repo](https://github.com/wesleyacheng/movie-review-sentiment-classifier-with-bert).